In [23]:
#!pip install simple_salesforce

In [24]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [25]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [26]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [27]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [28]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [29]:
staff_data_df.rename(columns={
    'ID_Staff':'ID_Staff__c',
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'BirthDate__c',
    
}, inplace=True)
staff_data_df.head()

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [30]:
staff_data_df = staff_data_df[['ID_Staff__c', 'EmployeeID__c', 'LastName__c', 'FirstName__c', 'MiddleName__c', 'BirthDate__c']]
staff_data_df.head()

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [31]:
staff_data_df["BirthDate__c"]=staff_data_df["BirthDate__c"].astype(str)

In [32]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'LastName__c': 'Wimberly',
  'FirstName__c': 'Sam',
  'MiddleName__c': 'Nico',
  'BirthDate__c': '1995-07-05'},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': '1992-04-22'},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': '1993-05-21'},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'LastName__c': 'Popelka',
  'FirstName__c': 'Sarah',
  'MiddleName__c': 'Nisan',
  'BirthDate__c': '1994-06-07'}]

In [33]:
for rec in staff_data_records:

    record = {
        'ID_Staff__c': rec['ID_Staff__c'],
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': rec['BirthDate__c'],
    }
    
    try:
        sf.Staff__C.create(record)
    except Exception as e:
        print(e)
           

In [37]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT ID_Staff__c, Name FROM Staff__c")
for row in data:
    rec = {
        'Staff__c': row['Name'],
        'ID_Staff__c': row['ID_Staff__c']
    }
    staff_lookup_list.append(rec)

In [38]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,Staff__c,ID_Staff__c
0,a0P3h000001dIxc,1
1,a0P3h000001dIy6,3
2,a0P3h000001dIxw,1
3,a0P3h000001dIxr,4
4,a0P3h000001dIxN,3
5,a0P3h000001dIy1,2
6,a0P3h000001dIxm,3
7,a0P3h000001dIyB,4
8,a0P3h000001dIxh,2
9,a0P3h000001dIxD,1


In [39]:
query = '''
    SELECT
        sa.*
        ,id.Staff
    FROM
        staffAssignment sa
        INNER JOIN staff st
        ON sa.ID_Staff = st.ID_Staff
'''
staff_data_df = pd.read_sql(query, conn)
staff_data_df.head()

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'gwsis.staffAssignment' doesn't exist")
[SQL: 
    SELECT
        sa.*
        ,id.Staff
    FROM
        staffAssignment sa
        INNER JOIN staff st
        ON sa.ID_Staff = st.ID_Staff
]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
staff_data_df.rename(columns={
    'ID_StaffAssignment':'ID_StaffAssignment__c',
    'ID_Staff':'ID_Staff__c',
    'ID_Class':'ID_Class__c',
    'Role':'Role__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
}, inplace=True)

staff_data_df

In [ ]:
staffassignment_data_df = pd.merge(staffassignment_data_df, staff_lookup_df, how='left')
staffassignment_data_df.head()

In [ ]:
staffassignment_data_df
staffassignment_data_df['Start_Date__c'] = pd.to_datetime(staff_data_df['Start_Date__c']).dt.date
staffassignment_data_df['End_Date__c'] = pd.to_datetime(staff_data_df['End_Date__c']).dt.date

staffAssignment_data_df.head()

In [ ]:
staffassignment_data_records = class_data_df.to_dict(orient='records')
class_data_records

In [ ]:
for rec in staffassignment_data_records:
 
    record = {
        'ID_StaffAssignment__c': rec['ID_StaffAssignment__c'],
        'ID_Staff__c': rec['ID_Staff__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'Role__c': rec['Role__c'],
        'StartDate__c': str(rec['Start_Date__c']),
        'EndDate__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.StaffAssignment__C.create(record)
    except Exception as e:
        print(e)